In [1]:
import os
import sys
import numpy as np
from PIL import Image
import imageio
from tqdm import tqdm
import shutil

In [2]:
base_dir = "./render/renderings"
depth_scale = 1000.0
cases = ["sony", "helmet", "marci", "miyuki", "pigeon", "pine_cone"]

## Rename the images

In [3]:
for case in cases:
    if not os.path.isdir(os.path.join(base_dir, case)):
        continue
    os.makedirs(os.path.join(base_dir, case, "images"), exist_ok=True)
    images = os.listdir(os.path.join(base_dir, case))
    for img_name in tqdm(images):
        if "normal000" in img_name or "depth000" in img_name or "mask000" in img_name:
            basename, ext = os.path.splitext(img_name)
            os.rename(
                os.path.join(base_dir, case, img_name),
                os.path.join(base_dir, case, "images", basename[:-4] + ext),
            )
            # print(f"Renamd {img_name} to images/{basename[:-4] + ext}")
        elif img_name.endswith("png"):
            img = Image.open(os.path.join(base_dir, case, img_name))
            img = img.convert("RGB")
            img.save(os.path.join(base_dir, case, "images", img_name.replace("png", "jpg")))
            os.rename(os.path.join(base_dir, case, img_name), os.path.join(base_dir, case, "images", img_name))
        elif img_name.endswith("exr") and not "vis" in img_name:
            img = imageio.imread(os.path.join(base_dir, case, img_name))
            img = img[..., :3]
            imageio.imsave(os.path.join(base_dir, case, img_name.replace(".exr", "_vis.exr")), img)
        # print(f"Skipping {im}")
        

  0%|          | 0/41 [00:00<?, ?it/s]/tmp/ipykernel_123486/2084426238.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(os.path.join(base_dir, case, img_name))
 22%|██▏       | 9/41 [00:00<00:00, 60.86it/s]/tmp/ipykernel_123486/2084426238.py:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(os.path.join(base_dir, case, img_name))
100%|██████████| 41/41 [00:00<00:00, 55.16it/s]


In [4]:
for case in cases:
    if not os.path.isdir(os.path.join(base_dir, case)):
        continue
    depth_imgs = [im for im in os.listdir(os.path.join(base_dir, case, "images")) if "depth.exr" in im]
    for d_name in tqdm(depth_imgs):
        d_im = imageio.imread(os.path.join(base_dir, case, "images", d_name))
        m_name = d_name.replace("depth.exr", "mask.png")
        m_im = imageio.imread(os.path.join(base_dir, case, "images", m_name))
        m_im = (m_im[..., None] / 255.0).astype(np.float32)
        d_im = d_im * m_im
        
        d_im = (d_im * depth_scale)[..., 0].astype(np.uint16)
        
        imageio.imwrite(os.path.join(base_dir, case, "images", d_name.replace("exr", "png")), d_im, extension="png")
        


  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipykernel_123486/1704216580.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  d_im = imageio.imread(os.path.join(base_dir, case, "images", d_name))
/tmp/ipykernel_123486/1704216580.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  m_im = imageio.imread(os.path.join(base_dir, case, "images", m_name))
100%|██████████| 5/5 [00:00<00:00,  9.42it/s]


In [5]:
def split_dataset(meta, train_ratio=0.6, val_ratio=0.2):
    meta_train, meta_val, meta_test = {}, {}, {}
    for key, value in meta.items():
        if key.startswith("frames"):
            frames = meta["frames"]
            for frame in frames:
                frame["file_path"] = "images/" + frame["file_path"]
            num_frames = len(frames)
            num_train = int(num_frames * train_ratio)
            num_val = int(num_frames * val_ratio)
            idx = np.random.permutation(num_frames)
            meta_train["frames"] = [frames[i] for i in idx[:num_train]]
            meta_val["frames"] = [frames[i] for i in idx[num_train : num_train + num_val]]
            meta_test["frames"] = [frames[i] for i in idx[num_train + num_val :]]
        else:
            meta_train[key] = value
            meta_val[key] = value
            meta_test[key] = value
    return meta_train, meta_val, meta_test

In [6]:
import json

In [7]:
for case in cases:
    if not os.path.isdir(os.path.join(base_dir, case)):
        continue
    with open(os.path.join(base_dir, case, "transforms.json"), "r") as f:
        meta = json.load(f)
    meta_train, meta_val, meta_test = split_dataset(meta)
    
    
    with open(os.path.join(base_dir, case, "transforms_train.json"), "w") as f:
        json.dump(meta_train, f, indent=4)

    with open(os.path.join(base_dir, case, "transforms_val.json"), "w") as f:
        json.dump(meta_val, f, indent=4)
        
    with open(os.path.join(base_dir, case, "transforms_test.json"), "w") as f:
        json.dump(meta_test, f, indent=4)

In [8]:
from arguments import ModelParams
from eval_metrics_rotblender import get_mesh_eval_points
from scene.gaussian_model import GaussianModel
from argparse import ArgumentParser, Namespace

from scene import Scene

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
for case in cases:
    cfg = {
        "model_path": "",
        "source_path": os.path.join(base_dir, case),
        "white_background": True,
        "eval": False,
        "rotation": True,
        "apply_mask": False,
        "linear": False,
        "shuffle": False,
        "resolution": 1,
        "data_device": "cuda"
    }
    cfg = Namespace(**cfg)
    gaussians = GaussianModel(3, -1, None, None)
    scene = Scene(cfg, gaussians, shuffle=False)

    get_mesh_eval_points(scene, source_dir=os.path.join(base_dir, case))

Found transforms_train.json file, assuming Blender data set!
Reading Training Transforms


FileNotFoundError: [Errno 2] No such file or directory: './render/renderings/sony/./render/renderings/sony/images/01_000.png'